In [92]:
import numpy as np
from ripser import ripser
from persim import plot_diagrams
import matplotlib.pyplot as plt
import math
import datetime as dt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

def logret(df):
    # Convert dataframe from daily close prices to daily log returns
    a = list(df.index)
    dfdict = df.T.apply(tuple).to_dict()
    return [[math.log(dfdict[a[i+1]][j]/dfdict[a[i]][j]) for j in range(0,len(dfdict[a[i]]))] for i in range(0,len(a)-1)]
def sliding_point_cloud_list(a,width):
    # Get sliding window point cloud from a list
    return [a[i:i+width] for i in range(0,len(a)-width)]
def sliding_point_cloud_dataframe(df,width):
    # Get sliding window point cloud from a dataframe
    a = list(df.index)
    dfdict = df.T.apply(tuple).to_dict()
    return [[dfdict[d] for d in a[i-width:i]] for i in range(width,len(a))]

import importlib
import landscapes
importlib.reload(landscapes)

def stonk_df(start_dt, end_dt, ticker_symbol_list):
    return pd.concat(
        [web.DataReader(sym, 'yahoo', start_dt, end_dt)['Close'] for sym in ticker_symbol_list],
        axis=1,
        keys=ticker_symbol_list
    )


In [32]:
df = stonk_df(
    start_dt=dt.datetime(2020, 1, 1),
    end_dt=dt.datetime(2020, 4, 29),
    ticker_symbol_list=["^GSPC","^DJI","^IXIC","^RUT"]
)

In [22]:
def compute_window_landscapes(start_dt, end_dt, ticker_symbol_list, window_size):
    df = stonk_df(start_dt = start_dt, end_dt = end_dt, ticker_symbol_list=ticker_symbol_list)
    lfd = logret(df)
    ldf = pd.DataFrame(lfd)
    sliding_4D = sliding_point_cloud_list(lfd,window_size)
    k = [np.nan] * (window_size + 1)
    k1 = [np.nan] * (window_size + 1)
    lands = []
    lands1 = []
    for i in range(len(sliding_4D)):
        dg = ripser(pd.DataFrame(sliding_4D[i]), maxdim=3)['dgms']
        land = landscapes.Landscape_Reader.read_fromlist(dg[0])
        lands += [land]
        land1 = landscapes.Landscape_Reader.read_fromlist(dg[1])
        lands1 += [land1]
        a = land.integrate()
        k += [a]
        b = land1.integrate()
        k1 += [b]
    df['L0'] = k
    df['L1'] = k1
    return df

In [200]:
def date_by_subtracting_nyse_calendar_days(from_date, num_days):
    import pandas_market_calendars as mcal
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.valid_days(start_date=from_date - dt.timedelta(days=2*num_days), end_date=from_date)
    calendar_days_to_subtract = num_days
    current_date = from_date
    while calendar_days_to_subtract > 0:
        current_date -= dt.timedelta(days=1)
        if current_date in schedule:
            calendar_days_to_subtract -= 1
    return current_date

In [246]:
def plot_window_landscapes(start_dt, end_dt, ticker_symbol_list, window_size, row_syms=["^GSPC","L0","L1"],highlight_dates=[],savefig=None):
    from dateutil.parser import isoparse
    start_dt = date_by_subtracting_nyse_calendar_days(isoparse(start_dt),window_size)
    end_dt = isoparse(end_dt)
    highlight_dates = [isoparse(x) for x in highlight_dates]
    
    df = compute_window_landscapes(start_dt = start_dt, end_dt = end_dt, ticker_symbol_list=ticker_symbol_list, window_size= window_size)

    new_row_syms = [x for x in row_syms if x not in ticker_symbol_list and x not in ['L0','L1']]
    if len(new_row_syms) > 0:
        new_row_syms_df = stonk_df(start_dt=start_dt,end_dt=end_dt,ticker_symbol_list=new_row_syms)
        for row_sym in new_row_syms:
            df[row_sym] = new_row_syms_df[row_sym]
    
    fig, axes = plt.subplots(nrows=len(row_syms))
    
    width=window_size+1
    for ax, row_sym in zip(axes[:-1], row_syms[:-1]):
        ax.set_ylabel(row_sym)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        for dte in highlight_dates:
            ax.axvline(dte, alpha=0.5)
    axes[-1].set_ylabel(row_syms[-1])
    axes[-1].set_yticklabels([])
    xlabels = []
    for x in df.index[width:]:
        if isoparse(str(x)) in highlight_dates:
            xlabels.append(x)
        else:
            xlabels.append('')
    for dte in highlight_dates:
            if type(highlight_dates) == dict:
                axes[-1].axvline(dte, label=highlight_dates[dte], alpha=0.5)
            else:
                axes[-1].axvline(dte, label=dte, alpha=0.5)
    plt.setp(axes[-1].get_xticklabels(), rotation=30, horizontalalignment='right')
    
    colors = ('k', 'r', 'b')
    colors = [colors[i%3] for i in range(len(row_syms))]
    xs = [df.index[width:]] * len(row_syms)
    ys = []
    for sym in row_syms:
        if sym in ['L0','L1']:
            ys.append(df[sym][width:])
        else:
            ys.append(df[sym].iloc[width:])
    for ax, color, x, y in zip(axes, colors, xs, ys):
        ax.plot(x, y, color=color)
        
    if savefig != None:
        plt.savefig(savefig)
    plt.show()

In [ ]:
for w in [14,30,60]:
    plot_window_landscapes(
            start_dt='2020-01-01',
            end_dt='2020-04-30',
            ticker_symbol_list=["^GSPC","^DJI","^IXIC","^RUT"],
            window_size=w,
            row_syms=["^GSPC","VIXY","L1"],
            savefig='windowlands'+str(w)+'.png'
    )